In [1]:
# import os
# os.environ['https_proxy'] = "http://hpc-proxy00.city.ac.uk:3128"
# # os.environ['http_proxy'] = “http://hpc-proxy00.city.ac.uk:3128”

In [2]:
import numpy as np
from numpy.random import default_rng
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import PIL
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torch.utils.data.dataloader as dataloader
from torchvision import datasets
import torchvision.models as models

In [3]:
transform_train = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
transform_test = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),     
            ])

In [4]:
filepath='.'
filedir=os.path.join(os.getcwd(),filepath,'Landscape Classification')
batch_size=16

# dataset=datasets.ImageFolder(
#     root=filedir,
#     transform=transform_train,
# )

# train_dataset,test_dataset,valid_dataset=torch.utils.data.random_split(dataset, [0.8,0.1,0.1],
# generator=torch.Generator().manual_seed(42))

for i in os.listdir(filedir):
    if(i=='Training Data'):
        train_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_train
    )
    elif(i=='Testing Data'):
        test_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_test
    )
    elif(i=='Validation Data'):
        valid_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_test
    )
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
valid_loader=DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

# Base Model

In [5]:
D1=32
D2=64
D3=128

A1=512
A2=256
num_classes=5
input_pix=224
num_neurons=int(np.floor(np.floor(np.floor(input_pix/2)/2)/2)**2*D3)
input=3
class my_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(input, D1, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(D1, D2, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(D2, D3, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            
            
            nn.Linear(num_neurons,A1),
            nn.ReLU(),
            nn.Linear(A1, A2),
            nn.ReLU(),
            nn.Linear(A2,num_classes)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [7]:
#Loading checkpoint in case of outside problems interrupting training
mydir='MODELS'
save_path=os.path.join(mydir, 'mymodel.pt')

device = ('cuda' if torch.cuda.is_available() else 'cpu')
lr = 1e-3
epochs = 5
opt=torch.optim.Adam
model=my_nn().to(device)

if not os.path.isdir(mydir):
    os.makedirs(mydir)

if os.path.isfile(save_path):
    checkpoint=torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    opt.load_state_dict(checkpoint['optimizer_state_dict'])
    start=checkpoint['epoch']
    best_valid=checkpoint['valid_acc']
    best_test=checkpoint['test_acc']
    print('Starting from epoch', start)
else:
    print('No Checkpoint, starting from scratch')

checkpoint=torch.load

RuntimeError: Error(s) in loading state_dict for my_nn:
	Missing key(s) in state_dict: "network.0.weight", "network.0.bias", "network.3.weight", "network.3.bias", "network.6.weight", "network.6.bias", "network.10.weight", "network.10.bias", "network.12.weight", "network.12.bias", "network.14.weight", "network.14.bias". 
	Unexpected key(s) in state_dict: "features.0.weight", "features.0.bias", "features.3.weight", "features.3.bias", "features.6.weight", "features.6.bias", "features.8.weight", "features.8.bias", "features.10.weight", "features.10.bias", "classifier.1.weight", "classifier.1.bias", "classifier.4.weight", "classifier.4.bias", "classifier.6.weight", "classifier.6.bias". 

In [ ]:
def train(model,train_loader,epochs,lr,opt,loss_func):
    model.train()
    losslog=[]
    optimizer=opt
    for _,(x,y) in enumerate(train_loader):
        batch_x=x.to(device)
        ypred=model(batch_x)
        loss=loss_func(ypred,y.to(device))
        losslog.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return losslog

def evaluate(model,test_loader):
    model.eval()
    acc = 0
        
    with torch.no_grad():
        for _, (x, y) in enumerate(test_loader):
            batch_x = model(x.to(device))
            acc += (batch_x.argmax(1) == y.to(device)).sum().item()
            
    #return the accuracy from the epoch     
    return acc / len(test_loader.dataset)
            

def fit(model,train_loader,test_loader,val_loader,epochs,optimizer,lr=1e-3,loss_func=F.cross_entropy):
    train_acc_log=[]
    test_acc_log=[]
    valid_acc_log=[]
    best_valid=0
    best_test=0
    opt=optimizer(model.parameters(),lr)
    for iter in range(epochs):
        #training step
        train_log=train(model,train_loader,epochs,lr,opt,loss_func)
            
        train_acc=evaluate(model,train_loader)
        test_acc=evaluate(model,test_loader)
        valid_acc=evaluate(model,test_loader)
        print("Epoch [{}], train acc: {:.2f}, test acc: {:.2f}, val acc: {:.2f}".format(iter, train_acc, test_acc, valid_acc))
        #evaluate step
        train_acc_log.append(train_acc)
        test_acc_log.append(test_acc)
        valid_acc_log.append(valid_acc)
        
        #saving models to avoid running through entire epochs
        if(valid_acc>best_valid and test_acc>=best_test):
            best_valid=valid_acc
            best_test=test_acc
            
            torch.save(
                {
                    'epochs': epochs,
                    'model_state_dict': model.state_dict(),
                    'opt_state_dict': opt.state_dict(),
                    'train_acc': train_acc,
                    'test_acc': test_acc,
                    'valid_acc': valid_acc,
                }, save_path
            )
            print('Model Saved for epoch {}'.format(iter))
    
    
    print('Training Complete')
    return train_acc,test_acc,valid_acc



In [ ]:
train_acc,test_acc,valid_acc=fit(model,train_loader,test_loader,valid_loader,20,opt)

In [ ]:
# transfer_alexnet = models.alexnet(weights='DEFAULT').to(device)
# train_acc=evaluate(transfer_alexnet,train_loader)
# test_acc=evaluate(transfer_alexnet,test_loader)
# valid_acc=evaluate(transfer_alexnet,test_loader)
# print("Epoch [{}], train acc: {:.2f}, test acc: {:.2f}, val acc: {:.2f}".format(1, train_acc, test_acc, valid_acc))
# #evaluate step

        

In [ ]:
D1=32
D2=64
D3=128
D4=256
D5=512

A1=512
A2=256
num_classes=5
input_pix=224
num_neurons=int(np.floor(np.floor(np.floor(input_pix/2)/2)/2)**2*D5)
input=3
class my_nn2(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(input, D1, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(D1, D2, kernel_size = 3, padding = 1),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(D2, D3, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(D3, D4, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(D4, D5, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(D5, D5, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            
            
            nn.Linear(num_neurons,A1),
            nn.ReLU(),
            nn.Linear(A1, A2),
            nn.ReLU(),
            nn.Linear(A2,num_classes)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
#Loading checkpoint in case of outside problems interrupting training
mydir='MODELS'
save_path=os.path.join(mydir, 'mymodel2.pt')

device = ('cuda' if torch.cuda.is_available() else 'cpu')
lr = 1e-3
epochs = 5
opt=torch.optim.Adam
model=my_nn2().to(device)

if not os.path.isdir(mydir):
    os.makedirs(mydir)

if os.path.isfile(save_path):
    checkpoint=torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt.load_state_dict(checkpoint['optimizer_state_dict'])
    start=checkpoint['epoch']
    best_valid=checkpoint['valid_acc']
    best_test=checkpoint['test_acc']
    print('Starting from epoch', start)
else:
    print('No Checkpoint, starting from scratch')

checkpoint=torch.load

In [ ]:
train_acc,test_acc,valid_acc=fit(model,train_loader,test_loader,valid_loader,20,opt)